In [39]:
from bs4 import BeautifulSoup, Comment
import requests
import regex as re
import xml
from selenium import webdriver
import pandas as pd

In [40]:
website = ["https://www.kasikornbank.com/en/rate/pages/foreign-exchange.aspx",
"https://www.scb.co.th/th/personal-banking/foreign-exchange-rates.html",
"https://www.bangkokbank.com/th-TH/Personal/Other-Services/View-Rates/Foreign-Exchange-Rates",
"https://exchangerate.krungthai.com/",
"https://www.krungsri.com/th/exchangerate/Todayrates"]
#ธนาคารซื้อ #ธนาคารขาย

#Remove HTML T A G S by regex |||||||||| Actually, use get_text() Function. It's much easier. =,,=

In [41]:
def remove_tag(html_str):
    TAG_RE = re.compile(r'<[^>]+>')
    text = TAG_RE.sub('!@', html_str)
    mylist = re.split(r'!@', text)
    clear_lst = []
    for clear_text in mylist:
        if (clear_text not in "\n") and (clear_text not in " ") and (clear_text not in ""):
            clear_lst.append(clear_text)
    return clear_lst
#https://www.guru99.com/python-regular-expressions-complete-tutorial.html

#start browser

In [42]:
def start_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    # executable_path param is not needed if you updated PATH
    browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')
    return browser

def end_browser(browser):
    browser.quit()


#bank

In [43]:
def kasikorn(url): #33
    browser = start_browser()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    end_browser(browser)

    #clean soup data
    data = soup.find('div',{"id" : "divTableRate"}).find('tbody').find_all('tr')
    lst_rows = []
    for index,value in enumerate(data):
        if index%2==0:
            lst = remove_tag(str(value))
            lst2 = lst[2:]
            lst_buy = [lst2[3],lst2[1],lst2[0],"-",lst2[2]]
            lst_sell = [lst2[5],lst2[4],lst2[4],lst2[4]]
            lst_rows.append([lst[0].replace(" ",""),lst_buy,lst_sell])
    
    return lst_rows

In [44]:
def scb(url): #29
    browser = start_browser()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    end_browser(browser)
    
    #clean soup data
    data = soup.find('div',{"class" : "exchange-rate"}).find('tbody').find_all('tr')
    data = data[2:]
    lst_rows = []
    for index,value in enumerate(data):
        lst = remove_tag(str(value))
        lst2 = lst[2:]
        lst_buy = [lst2[5],lst2[3],lst2[4],"-",lst2[2]]
        lst_sell = [lst2[1],lst2[0],lst2[0],"-"]
        lst_rows.append([lst[0],lst_buy,lst_sell])
    return lst_rows
    

In [45]:
def bangkok(url):
    browser = start_browser()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    end_browser(browser)

    #clean soup data
    data = soup.find('table',{"class" : "table-primary table-foreign-exchange-rates blue"}).find('tbody').find_all('tr')
    lst_rows = []
    for index,value in enumerate(data):
        clear_lst = remove_tag(str(value))
        clear_lst = [x.replace(" ", "") for x in clear_lst]
        while(len(clear_lst) < 7):
            clear_lst.append('-')
        lst2 = clear_lst[2:]
        lst_buy = [lst2[0],'-','-',lst2[2],lst2[3]]
        lst_sell = [lst2[1],"-",lst2[4],lst2[4]]
        lst_rows.append([clear_lst[0],lst_buy,lst_sell])
    return lst_rows

In [191]:
def krungthai(url):
    browser = start_browser()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    end_browser(browser)

    #clean soup data
    data = soup.find('table',{"id" : "cur-table-1"}).find('tbody').find_all('tr',{"class" : "ng-scope"})
    lst_rows = []
    for index,value in enumerate(data):
        if index != 8 :
            clear_lst = remove_tag(str(value))
            clear_lst = [x.replace(" ", "") for x in clear_lst]
            clear_lst = [x.replace("Unq", "-") for x in clear_lst]
            lst2 = clear_lst[2:]
            lst_buy = [lst2[3],lst2[0],'-','-',lst2[1]]
            lst_sell = [lst2[4],lst2[2],'-','-']
            lst_rows.append([clear_lst[0],lst_buy,lst_sell])
    return lst_rows

In [192]:
def krungsri(url):
    browser = start_browser()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, features="html.parser")
    end_browser(browser)

    #clen soup data
    data = soup.find('div',{"class" : "table-scroll"}).find('tbody').find_all('tr')
    lst_rows = []
    for index,value in enumerate(data):
        lst = remove_tag(str(value))
        remove = re.compile(r'\s|\xa0|\*')
        lst = [remove.sub('', x) for x in lst]
        lst = [x.replace("UNQ.", "-") for x in lst]
        lst2 = lst[-5:]
        lst_buy = [lst2[0],lst2[2],'-','-',lst2[3]]
        lst_sell = [lst2[1],lst2[4],lst2[4],lst2[4]]
        lst_rows.append([lst[1],lst_buy,lst_sell])
    return lst_rows

#start browser

In [46]:
kasi_data = kasikorn(website[0])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12516\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [47]:
scb_data = scb(website[1])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12516\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [48]:
bangkok_data = bangkok(website[2])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12516\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [193]:
krungthai_data = krungthai(website[3])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12516\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [194]:
krungsri_data = krungsri(website[4])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12516\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [186]:
kasi_data

[['USD1',
  ['32.4613', '33.37', '33.37', '-', '33.47'],
  ['33.81', '33.77', '33.77', '33.77']],
 ['USD5-20',
  ['32.72386', '33.37', '33.37', '-', '33.47'],
  ['33.86', '33.77', '33.77', '33.77']],
 ['USD50-100',
  ['33.21616', '33.37', '33.37', '-', '33.47'],
  ['33.886', '33.77', '33.77', '33.77']],
 ['AED',
  ['7.74648', '-', '-', '-', '8.75238'],
  ['9.63057', '9.5787', '9.5787', '9.5787']],
 ['AUD',
  ['24.39612', '24.54518', '24.54518', '-', '24.62485'],
  ['25.73309', '25.59358', '25.59358', '25.59358']],
 ['BHD', ['74.0825', '-', '-', '-', '-'], ['94.52125', '-', '-', '-']],
 ['BND',
  ['23.71073', '24.29833', '-', '-', '24.38833'],
  ['24.96974', '25.15787', '25.15787', '25.15787']],
 ['CAD',
  ['26.00297', '26.34701', '26.34701', '-', '26.42179'],
  ['27.14338', '27.00909', '27.00909', '27.00909']],
 ['CHF',
  ['35.17638', '35.55819', '35.55819', '-', '35.66605'],
  ['36.42569', '36.34893', '36.34893', '36.34893']],
 ['CNY',
  ['4.91749', '5.16553', '-', '-', '5.21429'],
  

In [187]:
scb_data

[['USD',
  ['33.24', '33.39', '33.29', '-', '33.49'],
  ['33.94', '33.79', '33.79', '-']],
 ['USD1',
  ['32.76', '33.39', '33.29', '-', '33.49'],
  ['33.87', '33.79', '33.79', '-']],
 ['USD2',
  ['32.4', '33.39', '33.29', '-', '33.49'],
  ['33.85', '33.79', '33.79', '-']],
 ['EUR',
  ['36.30125', '36.49375', '36.36375', '-', '36.58375'],
  ['37.3775', '37.30625', '37.30625', '-']],
 ['GBP',
  ['43.47125', '43.69625', '43.54625', '-', '43.94625'],
  ['45.13625', '44.81375', '44.81375', '-']],
 ['JPY',
  ['27.07', '27.21', '27.11', '-', '27.33'],
  ['28.14', '28.115', '28.115', '-']],
 ['SGD',
  ['24.27625', '24.3875', '24.3175', '-', '24.4775'],
  ['25.215', '25.10375', '25.10375', '-']],
 ['HKD',
  ['4.1625', '4.22125', '4.19125', '-', '4.25125'],
  ['4.4275', '4.35625', '4.35625', '-']],
 ['KRW', ['0.023', '-', '-', '-', '-'], ['0.028', '-', '-', '-']],
 ['CHF',
  ['35.14875', '35.61', '35.52', '-', '35.71'],
  ['36.53125', '36.45625', '36.45625', '-']],
 ['AUD',
  ['24.47875', '24.56

In [188]:
bangkok_data

[['USD1', ['32.30', '-', '-', '-', '-'], ['33.84', '-', '-', '-']],
 ['USD5', ['32.65', '-', '-', '-', '-'], ['33.84', '-', '-', '-']],
 ['USD50',
  ['33.12', '-', '-', '33.34', '33.44'],
  ['33.84', '-', '33.74', '33.74']],
 ['GBP',
  ['43.39', '-', '-', '43.75750', '43.87250'],
  ['44.58', '-', '44.77500', '44.77500']],
 ['EUR',
  ['36.24', '-', '-', '36.44500', '36.53250'],
  ['37.22', '-', '37.26750', '37.26750']],
 ['JPY',
  ['27.11', '-', '-', '27.33000', '27.39500'],
  ['27.88', '-', '28.06000', '28.06000']],
 ['HKD',
  ['4.19', '-', '-', '4.23750', '4.25250'],
  ['4.34', '-', '4.33250', '4.33250']],
 ['MYR',
  ['7.32', '-', '-', '7.82000', '7.85750'],
  ['8.06', '-', '8.06000', '8.06000']],
 ['SGD',
  ['24.19', '-', '-', '24.35250', '24.42000'],
  ['25.09', '-', '25.08750', '25.08750']],
 ['BND', ['23.68', '-', '-', '-', '-'], ['24.89', '-', '-', '-']],
 ['CNY',
  ['4.95', '-', '-', '5.13750', '5.18000'],
  ['5.35', '-', '5.35250', '5.35250']],
 ['IDR',
  ['2.01', '-', '-', '2.

In [195]:
krungthai_data

[['USD', ['32.43', '33.38', '-', '-', '33.48'], ['33.85', '33.78', '-', '-']],
 ['USD', ['32.83', '-', '-', '-', '-'], ['33.85', '-', '-', '-']],
 ['USD', ['33.23', '-', '-', '-', '-'], ['33.85', '-', '-', '-']],
 ['EUR',
  ['36.39', '36.38649', '-', '-', '36.49899'],
  ['37.33', '37.32335', '-', '-']],
 ['GBP',
  ['43.57', '43.7356', '-', '-', '43.8656'],
  ['44.77', '44.7399', '-', '-']],
 ['JPY',
  ['0.2707', '0.27097', '-', '-', '0.27285'],
  ['0.2818', '0.28099', '-', '-']],
 ['HKD',
  ['4.20', '4.24503', '-', '-', '4.25878'],
  ['4.37', '4.33823', '-', '-']],
 ['AUD', ['24.49', '24.57', '-', '-', '24.64'], ['25.69', '25.59', '-', '-']],
 ['SGD',
  ['24.30', '24.4276', '-', '-', '24.5014'],
  ['25.22', '25.0704', '-', '-']],
 ['CAD', ['26.12', '26.38', '-', '-', '26.46'], ['27.03', '27.03', '-', '-']],
 ['CNY', ['5.01', '5.1587', '-', '-', '5.1935'], ['5.45', '5.3682', '-', '-']],
 ['KHR', ['-', '-', '-', '-', '-'], ['-', '0.0083490', '-', '-']],
 ['DKK', ['4.71', '4.902', '-', '-

In [197]:
krungsri_data[-5]

['PHP',
 ['-', '-', '-', '-', '0.62663'],
 ['-', '0.65663', '0.65663', '0.65663']]

#test bank

In [101]:
browser = start_browser()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12516\1252230772.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='99.0.4844.51/chromedriver_win32/chromedriver.exe')


In [102]:
browser.get(website[4])
html = browser.page_source
soup = BeautifulSoup(html, features="html.parser")
end_browser(browser)

In [ ]:
# my_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36"}

# res = requests.get('https://exchangerate.krungthai.com/',headers=my_headers)
# soup = BeautifulSoup(res, 'html.parser')

#test

In [139]:
#clen soup data
data = soup.find('div',{"class" : "table-scroll"}).find('tbody').find_all('tr')
lst_rows = []
for index,value in enumerate(data):
    lst = remove_tag(str(value))
    clear_lst = []
    remove = re.compile(r'\s|\xa0|\*')
    for x in lst:
        x = remove.sub('', x)
        clear_lst.append(x)
    lst_rows.append(clear_lst)

In [143]:
data = soup.find('div',{"class" : "table-scroll"}).find('tbody').find_all('tr')
lst_rows = []
for index,value in enumerate(data):
    lst = remove_tag(str(value))
    remove = re.compile(r'\s|\xa0|\*')
    lst = [remove.sub('', x) for x in lst]
    # lst2 = clear_lst[2:]
    # lst_buy = [lst2[3],lst2[0],'-','-',lst2[1]]
    # lst_sell = [lst2[4],lst2[2],'-','-']
    lst_rows.append(lst)

In [156]:
#lst_rows
da = lst_rows[5][-5:]

In [159]:
da[2]

'27.29000'

In [29]:
filename = str('hello')+'.txt'
f = open(filename, 'w')
with open(filename, 'w',encoding='utf-8') as f:
    f.write(str(soup))

#pack data to csv file

#แยกเป็น แต่ละสกุลเงิน

In [158]:
buy_rate = ['สกุลเงิน','ธนบัตร', 'ตั๋วเงิน', 'เช็คเดินทาง','ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
br = pd.DataFrame()
br = br.reindex(columns = buy_rate)

In [159]:
sell_rate = ['สกุลเงิน','ธนบัตร', 'เช็คเดินทาง', 'ตั๋วแลกเงิน&ดราฟ','โอนเงินทางโทรเลข/โอนเงิน']
sr = pd.DataFrame()
sr = sr.reindex(columns = sell_rate)

In [160]:
br

,สกุลเงิน,ธนบัตร,ตั๋วเงิน,เช็คเดินทาง,ตั๋วแลกเงิน&ดราฟ,โอนเงินทางโทรเลข/โอนเงิน


In [161]:
sr

,สกุลเงิน,ธนบัตร,เช็คเดินทาง,ตั๋วแลกเงิน&ดราฟ,โอนเงินทางโทรเลข/โอนเงิน


In [108]:
for i,d in enumerate(data):
    dd = 
    df.loc[len(df)] = dd

In [109]:
df

,Date,Price,Open,High,Low,Change(%)
0,"Mar 18, 2022",33.290,33.210,33.400,33.195,0.27%
1,"Mar 17, 2022",33.200,33.400,33.400,33.180,-0.42%
2,"Mar 16, 2022",33.340,33.540,33.560,33.250,-0.51%
3,"Mar 15, 2022",33.510,33.450,33.570,33.420,0.18%
4,"Mar 14, 2022",33.450,33.300,33.470,33.270,0.45%
5,"Mar 11, 2022",33.300,33.140,33.360,33.115,0.54%
6,"Mar 10, 2022",33.120,33.040,33.170,32.970,0.30%
7,"Mar 09, 2022",33.020,33.220,33.235,32.975,-0.48%
8,"Mar 08, 2022",33.180,33.000,33.230,32.990,0.55%
9,"Mar 07, 2022",33.000,32.690,33.080,32.660,0.95%


In [110]:
df.to_csv('thb-usd.csv',index=False)

In [112]:
# f = open('text.txt', 'w')
# with open('text.txt', 'w',encoding='utf-8') as f:
#     f.write(str(soup))

In [113]:
#soup